In [1]:
from pyscf import gto, scf, dft
from pyscf.scf.hf import SCF, RHF
from pyscf.dft.rks import KohnShamDFT
import numpy as np

mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.0  0.0  0.0
H  0.0  0.7  1.0
"""
mol.basis = "6-31G"
mol.charge = 0
mol.multiplicity = 1
mol.verbose = 0
mol.unit = 'A'
mol.build()

mf = scf.HF(mol)
mf.kernel()

/usr/local/Caskroom/miniconda/base/lib/python3.11/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


-150.58503378083833

In [13]:
print(mf.mol)
print(mf.mo_occ)
mo = mf.mo_coeff[:,mf.mo_occ>0]
mo.shape

[2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


(22, 9)

In [3]:
from pyscf import dft

grids = dft.Grids(mol)
grids.prune = True
grids.atom_grid = (75, 302)
grids.becke_scheme
grids.build()
grids.coords
grids.weights

array([ 1.5390297 ,  2.03346446,  2.76800778, ..., 26.97613462,
        7.41030939,  3.58428041])

In [4]:
from pyscf.ita.promolecule import ProMolecule
from pyscf.ita.dens import ElectronDensity

moldens = ElectronDensity.build(mf, grids.coords, deriv=2)

promol = ProMolecule.build(mf)
promoldens = promol.electron_density(grids.coords, deriv=2)


omega = [free_atom_dens.density()/promoldens.density(mask=True) for free_atom_dens in promoldens]
omega

[masked_array(data=[0.10641337474454587, 0.024886912892937214,
                    0.0035205811319592916, ..., --, --, --],
              mask=[False, False, False, ...,  True,  True,  True],
        fill_value=1e+20),
 masked_array(data=[2.033826015569823e-09, 1.6045017484500194e-10,
                    6.535001110506969e-12, ..., --, --, --],
              mask=[False, False, False, ...,  True,  True,  True],
        fill_value=1e+20),
 masked_array(data=[0.8826260061671842, 0.966195039706771,
                    0.9899817499329502, ..., --, --, --],
              mask=[False, False, False, ...,  True,  True,  True],
        fill_value=1e+20),
 masked_array(data=[0.010960617054443796, 0.008918047239841466,
                    0.006497668928555538, ..., --, --, --],
              mask=[False, False, False, ...,  True,  True,  True],
        fill_value=1e+20)]

In [12]:
def relative_fisher_information(grids_weights, omega, moldens, promoldens):
    total = 0.0
    for i, omega_i in enumerate(omega):
        rho = moldens.density(mask=True)[np.newaxis,:]
        prorho = promoldens[i].density(mask=True)[np.newaxis,:]

        a = (moldens.gradient()/rho)
        b = (promoldens[i].gradient()/prorho)
        c = np.linalg.norm(a-b, axis=0)**2

        kernel = moldens.density()*omega_i*c

        d = (grids_weights * kernel).sum()
        print(d)
        total = total + d
    print(total)
    return total

def relative_alternative_fisher_information(grids_weights, omega, moldens, promoldens):
    total = 0.0
    for _, omega_i in enumerate(omega):
        rho = moldens.density()
        prorho = promoldens.density(mask=True)    

        kernel = moldens.laplacian()*omega_i*np.ma.log(rho/prorho)

        c = (grids_weights * kernel).sum()
        print(c)
        total +=c

    print(total)
    return total

def G2(grids_weights, omega, moldens, promoldens):
    total = 0.
    for _, omega_i in enumerate(omega):
        a = (moldens.laplacian()/moldens.density(mask=True))
        b = (promoldens.laplacian()/promoldens.density(mask=True))

        kernel = moldens.density()*omega_i*(a-b)

        d = (grids_weights * kernel).sum()
        print(d)
        total = total + d

    print(total)
    return total

relative_fisher_information(grids.weights, omega, moldens, promoldens)
print("*"*20)
#G1_function(grids.weights, omega, moldens, promol_dens)
relative_alternative_fisher_information(grids.weights, omega, moldens, promoldens)
print("*"*20)

#G1(grids.weights, omega, moldens, promol_dens, proatom_dens)
G2(grids.weights, omega, moldens, promoldens)


4.726641555262969
5.086179660002727
3.026751166766153
3.9195370232701765
16.759109405302027
********************
0.08319775262458665
-0.3425156445442316
-0.803871265123158
-0.7229910761129875
-1.7861802331557906
********************
-1.0777847623221917
-1.7437226107194514
0.5656653646436954
0.5059641917910251
-1.7498778166069229


-1.7498778166069229